In [ ]:
import pandas as pd
import os

# Pasta onde estão os arquivosssss
pasta_dados = r"C:\Users\Enrico\Documents\dados"

# Lista de arquivos a serem carregadossss
arquivos = [
    "vendas_linha_petshop_2019.csv",
    "vendas_linha_petshop_2020.csv",
    "vendas_linha_petshop_2021.csv",
    "vendas_linha_petshop_2022.csv"
]

# Função para carregar csv com tratamento de encoding
def carregar_csv(caminho):
    if not os.path.exists(caminho):
        print(f" Arquivo não encontrado: {caminho}")
        return None
    try:
        return pd.read_csv(caminho, sep=";", encoding="utf-8")
    except UnicodeDecodeError:
        return pd.read_csv(caminho, sep=";", encoding="latin-1")

# Carregar arquivos
dataframes = []
for arquivo in arquivos:
    caminho_completo = os.path.join(pasta_dados, arquivo)
    df_temp = carregar_csv(caminho_completo)
    if df_temp is not None:
        # Adicionar coluna do ano
        ano = int(arquivo.split("_")[-1].split(".")[0])
        df_temp["Ano"] = ano
        dataframes.append(df_temp)

# Verificar se carregou pelo menos um arquivo
if not dataframes:
    raise ValueError("Nenhum arquivo foi carregado. Verifique os nomes e caminhos dos CSVs.")

# Unir todos os anos em um único DataFrame
df = pd.concat(dataframes, ignore_index=True)

# Limpeza da coluna quantidade
df['quantidade'] = pd.to_numeric(df['quantidade'], errors='coerce')
df.dropna(subset=['quantidade'], inplace=True)
df['quantidade'] = df['quantidade'].astype(int)

# Estatísticas básicas
print("\nResumo estatístico da coluna QUANTIDADE:")
print(df["quantidade"].describe())

# Detectar outliers usando IQR (Intervalo Interquartil)
Q1 = df['quantidade'].quantile(0.25)
Q3 = df['quantidade'].quantile(0.75)
IQR = Q3 - Q1
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

outliers = df[(df['quantidade'] < limite_inferior) | (df['quantidade'] > limite_superior)]

print("\n--- Análise de Outliers ---")
print(f"Q1: {Q1}, Q3: {Q3}, IQR: {IQR}")
print(f"Limite inferior: {limite_inferior}, Limite superior: {limite_superior}")
print(f"Número de outliers: {outliers.shape[0]}")
print(f"Porcentagem de outliers: {outliers.shape[0]/df.shape[0]*100:.2f}%")


# calculo de desvio padrão com e sem outliers
std_original = df['quantidade'].std()
df_sem_outliers = df[(df['quantidade'] >= limite_inferior) & (df['quantidade'] <= limite_superior)]
std_sem_outliers = df_sem_outliers['quantidade'].std()

print("\n--- Comparação de Variabilidade ---")
print(f"Desvio padrão com outliers: {std_original:.2f}")
print(f"Desvio padrão sem outliers: {std_sem_outliers:.2f}")


Resumo estatístico da coluna QUANTIDADE:
count    200119.000000
mean          3.650638
std          12.045057
min           0.000000
25%           1.000000
50%           1.000000
75%           2.000000
max         110.000000
Name: quantidade, dtype: float64

--- Análise de Outliers ---
Q1: 1.0, Q3: 2.0, IQR: 1.0
Limite inferior: -0.5, Limite superior: 3.5
Número de outliers: 11283
Porcentagem de outliers: 5.64%

--- Comparação de Variabilidade ---
Desvio padrão com outliers: 12.05
Desvio padrão sem outliers: 0.78


In [9]:
import os
print(os.getcwd())

c:\Users\Enrico\Documents
